In [ ]:
import numpy as np
import torch
import torch.nn as nn
from mmcv.cnn import ConvModule, DepthwiseSeparableConvModule
from mmcv.runner import BaseModule
from mmdet.models.necks import YOLOXPAFPN
from mmdet.models.backbones import Darknet, CSPDarknet,MobileNetV2

from mmdet.models.dense_heads import YOLOXHead

from math import sqrt

from thop import profile,clever_format


In [ ]:
img_scale = (480,640) # height, width
test_backbone = CSPDarknet(
    arch='P4',
    deepen_factor=0.5,
    widen_factor=0.25,
    out_indices=(3,),
    use_depthwise=False,
    spp_kernal_sizes=(5, 9, 13),
    act_cfg=dict(type='LeakyReLU', negative_slope=0.1))
test_neck = YOLOXPAFPN(
    in_channels=[128,],
    out_channels=128,
    num_csp_blocks=2,
    use_depthwise=False,
    upsample_cfg=dict(scale_factor=2, mode='bilinear'),
    conv_cfg=None,
    norm_cfg=dict(type='BN', momentum=0.03, eps=0.001),
    act_cfg=dict(type='LeakyReLU', negative_slope=0.1),
   )
head = YOLOXHead(
    num_classes=8,
    in_channels=128,
    feat_channels=128,
    strides=[8,],
    use_depthwise=True,
    act_cfg=dict(type='LeakyReLU', negative_slope=0.1),
)
img_inputs = torch.randn(1, 3, img_scale[0], img_scale[1])


class NaiveModel(nn.Module):
    def __init__(self, backbone, neck, head):
        super(NaiveModel, self).__init__()
        self.backbone = backbone
        self.neck = neck
        self.head = head

    def forward(self, x):
        x = self.backbone(x)
        x = self.neck(x)
        x = self.head(x)
        return x


m = NaiveModel(
        test_backbone,
        test_neck,
        head
)
img_inputs = torch.randn(1, 3, img_scale[0], img_scale[1])
outputs = m(img_inputs)
print(outputs[0][0].shape)
torch.onnx.export(m, img_inputs, "litedbg.onnx", verbose=True, opset_version=11)

ops, params = profile(m, inputs=(img_inputs,))
print('model  #flops: {},\t#params: {}'.format(*clever_format([ops, params], "%.3f")))